# Monitor Model using Lakehouse Monitoring
In this step, we will leverage Databricks Lakehouse Monitoring([AWS](https://docs.databricks.com/en/lakehouse-monitoring/index.html)|[Azure](https://learn.microsoft.com/en-us/azure/databricks/lakehouse-monitoring/)) to monitor our inference table.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/mlops/advanced/banners/mlflow-uc-end-to-end-advanced-7.png?raw=true" width="1200">

Databricks Lakehouse Monitoring attaches a data monitor to any Delta table and it will generate the necessary pipelines to profile the data and calculate quality metrics. You just need to tell it how frequently these quality metrics need to be collected.

Use Databricks Lakehouse Monitoring to monitor for data drifts, as well as label drift, prediction drift and changes in model quality metrics in Machine Learning use cases. Databricks Lakehouse Monitoring enables monitoring for statistics (e.g. data profiles) and drifts on tables containing:
* batch scoring inferences
* request logs from Model Serving endpoint ([AWS](https://docs.databricks.com/en/machine-learning/model-serving/inference-tables.html) |[Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/model-serving/inference-tables))

Databricks Lakehouse Monitoring stores the data quality and drift metrics in two tables that it automatically creates for each monitored table:
- Profile metrics table (with a `_profile_metrics` suffix)
  - Metrics like percentage of null values, descriptive statistics, model metrics such as accuracy, RMSE, fairness and bias metrics etc.
- Drift metrics table (with a `_drift_metrics` suffix)
  - Metrics like the "delta" between percentage of null values, averages, as well as metrics from statistical tests to detect data drift.

For demo simplicity purpose, we will use the batch scoring model inference as our inference table. We will attach a monitor to the table `mlops_churn_advanced_inference_table`.


### A cluster has been created for this demo
To run this demo, just select the cluster `dbdemos-mlops-end2end-edgar_aguilerarod` from the dropdown menu ([open cluster configuration](https://dbc-07122dbb-1c85.cloud.databricks.com/#setting/clusters/0102-173414-9ev1v92w/configuration)). <br />
*Note: If the cluster was deleted after 30 days, you can re-create it with `dbdemos.create_cluster('mlops-end2end')` or re-install the demo: `dbdemos.install('mlops-end2end')`*

In [0]:
%pip install -qU "databricks-sdk>=0.28.0"
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-setup $reset_all_data=false $adv_mlops=true

### Create monitor
Now, we will create a monitor on top of the inference table. 
It is a one-time setup.

### Create Inference Table

This can serve as a union for offline & online processed inference.
For simplicity of this demo, we will create the inference table as a copy of the first offline batch prediction table.

In a different scenario, we could have processed the online inference table and store them in the inference table alongside with the offline inference table.

In [0]:
%sql
CREATE OR REPLACE TABLE advanced_churn_inference_table AS
          SELECT * EXCEPT (split) FROM advanced_churn_offline_inference LEFT JOIN advanced_churn_label_table USING(customer_id, transaction_ts) ;

ALTER TABLE advanced_churn_inference_table SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

### Create baseline table

For simplification purposes, we will create the baseline table from the pre-existing `advanced_churn_offline_inference` table

In [0]:
%sql
-- TODO: understand why we need model version in the baseline table
CREATE OR REPLACE TABLE advanced_churn_baseline AS
  SELECT * EXCEPT (customer_id, transaction_ts, model_alias, inference_timestamp) FROM advanced_churn_inference_table

### Create a custom metric

Customer metrics can be defined and will automatically be calculated by lakehouse monitoring. They often serve as a mean to capture some aspect of business logic or use a custom model quality score. 

In this example, we will calculate the business impact (loss in monthly charges) of a bad model performance

In [0]:
from pyspark.sql.types import DoubleType, StructField
from databricks.sdk.service.catalog import MonitorMetric, MonitorMetricType

expected_loss_metric = [
  MonitorMetric(
    type=MonitorMetricType.CUSTOM_METRIC_TYPE_AGGREGATE,
    name="expected_loss",
    input_columns=[":table"],
    definition="""avg(CASE
    WHEN {{prediction_col}} != {{label_col}} AND {{label_col}} = 'Yes' THEN -monthly_charges
    ELSE 0 END
    )""",
    output_data_type= StructField("output", DoubleType()).json()
  )
]

### Create monitor

As we are monitoring an inference table (including machine learning model predcitions data), we will pick an [Inference profile](https://learn.microsoft.com/en-us/azure/databricks/lakehouse-monitoring/create-monitor-api#inferencelog-profile) for the monitor.

In [0]:
import os
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.catalog import MonitorInferenceLog, MonitorInferenceLogProblemType

print(f"Creating monitor for inference table {catalog}.{db}.advanced_churn_inference_table")
w = WorkspaceClient()

info = w.quality_monitors.create(
  table_name=f"{catalog}.{db}.advanced_churn_inference_table",
  inference_log=MonitorInferenceLog(
        problem_type=MonitorInferenceLogProblemType.PROBLEM_TYPE_CLASSIFICATION,
        prediction_col="prediction",
        timestamp_col="inference_timestamp",
        granularities=["1 day"],
        model_id_col="model_version",
        label_col="churn", # optional
  ),
  assets_dir=f"{os.getcwd()}/monitoring", # Change this to another folder of choice if needed
  output_schema_name=f"{catalog}.{db}",
  baseline_table_name=f"{catalog}.{db}.advanced_churn_baseline",
  slicing_exprs=["senior_citizen='Yes'", "contract"], # Slicing dimension
  custom_metrics=expected_loss_metric
)

Wait/Verify that monitor was created

In [0]:
import time
from databricks.sdk.service.catalog import MonitorInfoStatus, MonitorRefreshInfoState

# Wait for monitor to be created
while info.status == MonitorInfoStatus.MONITOR_STATUS_PENDING:
  info = w.quality_monitors.get(table_name=f"{catalog}.{db}.advanced_churn_inference_table")
  time.sleep(10)

assert info.status == MonitorInfoStatus.MONITOR_STATUS_ACTIVE, "Error creating monitor"

Monitor creation for the first time will also **trigger an initial refresh** so fetch/wait or trigger a monitoring job and wait until completion

In [0]:
refreshes = w.quality_monitors.list_refreshes(table_name=f"{catalog}.{db}.advanced_churn_inference_table").refreshes
assert(len(refreshes) > 0)

run_info = refreshes[0]
while run_info.state in (MonitorRefreshInfoState.PENDING, MonitorRefreshInfoState.RUNNING):
  run_info = w.quality_monitors.get_refresh(table_name=f"{catalog}.{db}.advanced_churn_inference_table", refresh_id=run_info.refresh_id)
  time.sleep(30)

assert run_info.state == MonitorRefreshInfoState.SUCCESS, "Monitor refresh failed"

In [0]:
w.quality_monitors.get(table_name=f"{catalog}.{db}.advanced_churn_inference_table")

In [0]:
# w.quality_monitors.delete(table_name=f"{catalog}.{db}.advanced_churn_offline_inference", purge_artifacts=True)

## Inspect dashboard

You can now inspect the monitoring dashboard that is automatically generated for you. Navigate to `advanced_churn_inference_table` in the __Catalog Explorer__, go to the __Quality__ tab and click on the __View dashboard__ button.

<br>

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/mlops/advanced/07_view_dashboard_button.png?raw=true" width="480">

<br>

---

You can see the number of inferences being done before the first monitor refresh (the first refresh "window"), as well as the model performance metrics.

<br>

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/mlops/advanced/07_model_inferences.png?raw=true" width="1200">

<br>

Scrolling further down to the section on __Prediction drift__, you can see the confusion matrix and the percentage of the model's predictions.

<br>

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/mlops/advanced/07_confusion_matrix.png?raw=true" width="1200">

<br>

We do not observe any drift yet, as we only have the first refresh "window". We will simulate some drifted data in the next step and refresh the monitor against the newly captured data.

### Next: Test for drift and trigger a model retrain

Now, let explore how to detect drift on the inference data and define violations rules for triggering a model (re)train workflow.

Next steps:
* [Detect drift and trigger model retrain]($./08_drift_detection)